# TKAN example and comparison with benchmarks



tkan version: 0.4.1

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install pandas numpy matplotlib pyarrow scikit-learn tkan "jax[cuda12]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cuda-nvcc-cu12
    Found existing installation: nvidia-cuda-nvcc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvcc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvcc-cu12-12.5.82


In [2]:
import os
BACKEND = 'jax' # You can use any backend here
os.environ['KERAS_BACKEND'] = BACKEND

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Flatten, GRU

from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error

from tkan import TKAN

import time

keras.utils.set_random_seed(1)

N_MAX_EPOCHS = 1000
BATCH_SIZE = 128
early_stopping_callback = lambda : keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=10,
    mode="min",
    restore_best_weights=True,
    start_from_epoch=6,
)
lr_callback = lambda : keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=5,
    mode="min",
    min_delta=0.00001,
    min_lr=0.000025,
    verbose=0,
)
callbacks = lambda : [early_stopping_callback(), lr_callback(), keras.callbacks.TerminateOnNaN()]


# Data

In [5]:
df = pd.read_excel('data_2.xlsx')
df['Fecha'] = pd.to_datetime(df['Fecha'])
df.set_index('Fecha', inplace=True)
display(df)

,Precio,Hidráulica,Nuclear,Ciclo_combinado,Eólica,fotovoltaica,Demanda,Precio_gas,Laborable
Fecha,,,,,,,,,
2018-01-01,25.660,41.122351,170.287217,27.260011,245.674626,14.735956,539.925842,21.81,1
2018-01-02,40.896,52.697685,170.457684,30.520854,296.003932,12.771571,684.961951,22.51,1
2018-01-03,45.745,63.504680,168.512738,35.832860,280.929091,9.546451,713.422320,20.40,1
2018-01-04,45.370,64.629104,170.520780,28.449993,307.303065,8.969036,713.027277,19.01,1
2018-01-05,44.784,75.017331,170.732163,30.569147,232.172169,8.267151,685.084586,19.78,1
...,...,...,...,...,...,...,...,...,...
2025-04-06,26.559,120.495514,123.506343,53.506644,92.200415,117.842400,526.923889,36.12,0
2025-04-07,55.043,146.666168,145.193889,74.561048,65.279250,177.023599,619.184673,34.93,1
2025-04-08,47.672,151.905941,147.110434,64.227465,67.664256,159.110556,625.451271,35.05,1


In [6]:
class MinMaxScaler:
    def __init__(self, feature_axis=None, minmax_range=(0, 1)):
        """
        Initialize the MinMaxScaler.
        Args:
        feature_axis (int, optional): The axis that represents the feature dimension if applicable.
                                      Use only for 3D data to specify which axis is the feature axis.
                                      Default is None, automatically managed based on data dimensions.
        """
        self.feature_axis = feature_axis
        self.min_ = None
        self.max_ = None
        self.scale_ = None
        self.minmax_range = minmax_range # Default range for scaling (min, max)

    def fit(self, X):
        """
        Fit the scaler to the data based on its dimensionality.
        Args:
        X (np.array): The data to fit the scaler on.
        """
        if X.ndim == 3 and self.feature_axis is not None:  # 3D data
            axis = tuple(i for i in range(X.ndim) if i != self.feature_axis)
            self.min_ = np.min(X, axis=axis)
            self.max_ = np.max(X, axis=axis)
        elif X.ndim == 2:  # 2D data
            self.min_ = np.min(X, axis=0)
            self.max_ = np.max(X, axis=0)
        elif X.ndim == 1:  # 1D data
            self.min_ = np.min(X)
            self.max_ = np.max(X)
        else:
            raise ValueError("Data must be 1D, 2D, or 3D.")

        self.scale_ = self.max_ - self.min_
        return self

    def transform(self, X):
        """
        Transform the data using the fitted scaler.
        Args:
        X (np.array): The data to transform.
        Returns:
        np.array: The scaled data.
        """
        X_scaled = (X - self.min_) / self.scale_
        X_scaled = X_scaled * (self.minmax_range[1] - self.minmax_range[0]) + self.minmax_range[0]
        return X_scaled

    def fit_transform(self, X):
        """
        Fit to data, then transform it.
        Args:
        X (np.array): The data to fit and transform.
        Returns:
        np.array: The scaled data.
        """
        return self.fit(X).transform(X)

    def inverse_transform(self, X_scaled):
        """
        Inverse transform the scaled data to original data.
        Args:
        X_scaled (np.array): The scaled data to inverse transform.
        Returns:
        np.array: The original data scale.
        """
        X = (X_scaled - self.minmax_range[0]) / (self.minmax_range[1] - self.minmax_range[0])
        X = X * self.scale_ + self.min_
        return X

def generate_data(df, sequence_length, n_ahead = 1):
    #Case without known inputs
    scaler_df = df.copy().shift(n_ahead).rolling(24 * 14).median()
    tmp_df = df.copy() / scaler_df
    tmp_df = tmp_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    scaler_df = scaler_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    def prepare_sequences(df, scaler_df, n_history, n_future):
        X, y, y_scaler = [], [], []
        num_features = df.shape[1]

        # Iterate through the DataFrame to create sequences
        for i in range(n_history, len(df) - n_future + 1):
            # Extract the sequence of past observations
            X.append(df.iloc[i - n_history:i].values)
            # Extract the future values of the first column
            y.append(df.iloc[i:i + n_future,0:1].values)
            y_scaler.append(scaler_df.iloc[i:i + n_future,0:1].values)

        X, y, y_scaler = np.array(X), np.array(y), np.array(y_scaler)
        return X, y, y_scaler

    # Prepare sequences
    X, y, y_scaler = prepare_sequences(tmp_df, scaler_df, sequence_length, n_ahead)

    # Split the dataset into training and testing sets
    train_test_separation = int(len(X) * 0.8)
    X_train_unscaled, X_test_unscaled = X[:train_test_separation], X[train_test_separation:]
    y_train_unscaled, y_test_unscaled = y[:train_test_separation], y[train_test_separation:]
    y_scaler_train, y_scaler_test = y_scaler[:train_test_separation], y_scaler[train_test_separation:]

    # Generate the data
    X_scaler = MinMaxScaler(feature_axis=2)
    X_train = X_scaler.fit_transform(X_train_unscaled)
    X_test = X_scaler.transform(X_test_unscaled)

    y_scaler = MinMaxScaler(feature_axis=2)
    y_train = y_scaler.fit_transform(y_train_unscaled)
    y_test = y_scaler.transform(y_test_unscaled)

    y_train = y_train.reshape(y_train.shape[0], -1)
    y_test = y_test.reshape(y_test.shape[0], -1)
    return X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test



In [9]:
n_aheads = [1, 3, 6, 9, 12, 15]
models = [
    "TKAN",
    "GRU",
    "LSTM",
 ]

results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
results_rmse = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
time_results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
for n_ahead in n_aheads:
    sequence_length = max(45, 5 * n_ahead)
    X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test = generate_data(df, sequence_length, n_ahead)

    for model_id in models:

        for run in range(10):

            if model_id == 'TKAN':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    TKAN(100, return_sequences=True),
                    TKAN(100, sub_kan_output_dim = 20, sub_kan_input_dim = 20, return_sequences=False),
                    Dense(units=n_ahead, activation='linear')
                ], name = model_id)
            elif model_id == 'GRU':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    GRU(100, return_sequences=True),
                    GRU(100, return_sequences=False),
                    Dense(units=n_ahead, activation='linear')
                ], name = model_id)
            elif model_id == 'LSTM':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    LSTM(100, return_sequences=True),
                    LSTM(100, return_sequences=False),
                    Dense(units=n_ahead, activation='linear')
                ], name = model_id)
            else:
                raise ValueError

            optimizer = keras.optimizers.Adam(0.001)
            model.compile(optimizer=optimizer, loss='mean_squared_error', jit_compile=True)
            if run==0:
                model.summary()

            # Fit the model
            start_time = time.time()
            history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_MAX_EPOCHS, validation_split=0.2, callbacks=callbacks(), shuffle=True, verbose = False)
            end_time = time.time()
            time_results[model_id][n_ahead].append(end_time - start_time)
            # Evaluate the model on the test set
            preds = model.predict(X_test, verbose=False)
            r2 = r2_score(y_true=y_test, y_pred=preds)
            print(end_time - start_time, r2)
            rmse = root_mean_squared_error(y_true=y_test, y_pred=preds)
            results[model_id][n_ahead].append(r2)
            results_rmse[model_id][n_ahead].append(rmse)

            del model
            del optimizer


print('R2 scores')
print('Means:')
display(pd.DataFrame({model_id: {n_ahead: np.mean(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.mean(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse.keys()}))
print('Std:')
display(pd.DataFrame({model_id: {n_ahead: np.std(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.std(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse.keys()}))
print('Training Times')
display(pd.DataFrame({model_id: {n_ahead: np.mean(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.std(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results.keys()}))

Model: "TKAN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tkan_2 (TKAN)                   │ (None, 45, 100)        │        34,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tkan_3 (TKAN)                   │ (None, 100)            │        67,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,865 (401.82 KB)

 Trainable params: 102,575 (400.68 KB)

 Non-trainable params: 290 (1.13 KB)

...

,TKAN,GRU,LSTM
1,0.206027,0.007527,0.023752
3,0.028868,0.007343,0.017475
6,0.035305,0.003385,0.003609
9,0.105443,0.004164,0.013890
12,0.058856,0.004087,0.022453
15,0.043471,0.006354,0.021642


,TKAN,GRU,LSTM
1,0.021616,0.001324,0.003451
3,0.003589,0.001083,0.002219
6,0.003953,0.000458,0.000445
9,0.010621,0.000512,0.001609
12,0.006157,0.000518,0.002547
15,0.004441,0.000800,0.002380


Training Times


,TKAN,GRU,LSTM
1,327.232517,110.696955,142.642292
3,341.142383,93.562341,86.468775
6,248.170580,81.862028,77.960181
9,232.425442,75.904232,77.243937
12,228.429058,96.613552,91.243209
15,241.782171,104.943990,125.468138


,TKAN,GRU,LSTM
1,137.130785,14.076798,46.120193
3,201.157450,16.823181,17.235857
6,102.454691,12.653593,16.218031
9,81.708313,7.308373,20.362609
12,40.069492,12.443396,8.542839
15,59.320720,19.993833,25.266825


## Guardado de resultados

In [ ]:
output_prefix = "/content/drive/MyDrive/TFG_def"

import os
os.makedirs(os.path.dirname(output_prefix), exist_ok=True)

def export_results(results, results_rmse, output_prefix):
    model_ids = list(results.keys())
    n_aheads = list(results[model_ids[0]].keys())

    r2_means = pd.DataFrame({model_id: {n: np.mean(results[model_id][n]) for n in n_aheads} for model_id in model_ids})
    r2_stds = pd.DataFrame({model_id: {n: np.std(results[model_id][n]) for n in n_aheads} for model_id in model_ids})
    rmse_means = pd.DataFrame({model_id: {n: np.mean(results_rmse[model_id][n]) for n in n_aheads} for model_id in model_ids})
    rmse_stds = pd.DataFrame({model_id: {n: np.std(results_rmse[model_id][n]) for n in n_aheads} for model_id in model_ids})


    r2_means.to_csv(f"{output_prefix}_r2_means.csv")
    r2_stds.to_csv(f"{output_prefix}_r2_stds.csv")
    rmse_means.to_csv(f"{output_prefix}_rmse_means.csv")
    rmse_stds.to_csv(f"{output_prefix}_rmse_stds.csv")

    print(" Archivos guardados correctamente en tu Google Drive.")
    return r2_means, r2_stds, rmse_means, rmse_stds

# Ejecutar
r2_means, r2_stds, rmse_means, rmse_stds = export_results(results, results_rmse, output_prefix)

## Contraste de hipótesis: Wilcoxon test

In [32]:
from scipy.stats import wilcoxon


model_a = "TKAN"
model_b = "GRU"
n_ahead_list = [1, 3, 6, 9, 12, 15]

print(f"--- Wilcoxon test: {model_a.upper()} vs {model_b.upper()} ---\n")

for n_ahead in n_ahead_list:
    # Extrae los datos para R2 y RMSE
    r2_a = results[model_a][n_ahead]
    r2_b = results[model_b][n_ahead]
    rmse_a = results_rmse[model_a][n_ahead]
    rmse_b = results_rmse[model_b][n_ahead]

    # Calcula diferencias
    r2_diff = [a - b for a, b in zip(r2_a, r2_b)]
    rmse_diff = [a - b for a, b in zip(rmse_a, rmse_b)]

    # Wilcoxon unilateral para R2 (H1: TKAN > GRU)
    stat_r2, pval_r2 = wilcoxon(r2_diff, alternative='greater')

    # Wilcoxon unilateral para RMSE (H1: TKAN < GRU)
    stat_rmse, pval_rmse = wilcoxon(rmse_diff, alternative='less')

    print(f"Horizonte {n_ahead} días:")
    print(f"  R2    -> estadístico = {stat_r2:.3f}, p-valor = {pval_r2:.5f}")
    print(f"  RMSE  -> estadístico = {stat_rmse:.3f}, p-valor = {pval_rmse:.5f}")
    print()

--- Wilcoxon test: TKAN vs GRU ---

Horizonte 1 días:
  R2    -> estadístico = 0.000, p-valor = 1.00000
  RMSE  -> estadístico = 55.000, p-valor = 1.00000

Horizonte 3 días:
  R2    -> estadístico = 0.000, p-valor = 1.00000
  RMSE  -> estadístico = 55.000, p-valor = 1.00000

Horizonte 6 días:
  R2    -> estadístico = 0.000, p-valor = 1.00000
  RMSE  -> estadístico = 55.000, p-valor = 1.00000

Horizonte 9 días:
  R2    -> estadístico = 0.000, p-valor = 1.00000
  RMSE  -> estadístico = 55.000, p-valor = 1.00000

Horizonte 12 días:
  R2    -> estadístico = 0.000, p-valor = 1.00000
  RMSE  -> estadístico = 55.000, p-valor = 1.00000

Horizonte 15 días:
  R2    -> estadístico = 0.000, p-valor = 1.00000
  RMSE  -> estadístico = 55.000, p-valor = 1.00000



In [39]:
model_a = "TKAN"
model_b = "LSTM"
n_ahead_list = [1, 3, 6, 9, 12, 15]

print(f"--- Wilcoxon test: {model_a.upper()} vs {model_b.upper()} ---\n")

for n_ahead in n_ahead_list:
    # Extrae los datos para R2 y RMSE
    r2_a = results[model_a][n_ahead]
    r2_b = results[model_b][n_ahead]
    rmse_a = results_rmse[model_a][n_ahead]
    rmse_b = results_rmse[model_b][n_ahead]

    # Calcula diferencias
    r2_diff = [a - b for a, b in zip(r2_a, r2_b)]
    rmse_diff = [a - b for a, b in zip(rmse_a, rmse_b)]

    # Wilcoxon unilateral para R2 (H1: TKAN > GRU)
    stat_r2, pval_r2 = wilcoxon(r2_diff, alternative='greater')

    # Wilcoxon unilateral para RMSE (H1: TKAN < GRU)
    stat_rmse, pval_rmse = wilcoxon(rmse_diff, alternative='less')

    print(f"Horizonte {n_ahead} días:")
    print(f"  R2    -> estadístico = {stat_r2:.3f}, p-valor = {pval_r2:.5f}")
    print(f"  RMSE  -> estadístico = {stat_rmse:.3f}, p-valor = {pval_rmse:.5f}")
    print()

--- Wilcoxon test: TKAN vs LSTM ---

Horizonte 1 días:
  R2    -> estadístico = 17.000, p-valor = 0.86230
  RMSE  -> estadístico = 39.000, p-valor = 0.88379

Horizonte 3 días:
  R2    -> estadístico = 4.000, p-valor = 0.99512
  RMSE  -> estadístico = 49.000, p-valor = 0.99023

Horizonte 6 días:
  R2    -> estadístico = 0.000, p-valor = 1.00000
  RMSE  -> estadístico = 55.000, p-valor = 1.00000

Horizonte 9 días:
  R2    -> estadístico = 0.000, p-valor = 1.00000
  RMSE  -> estadístico = 55.000, p-valor = 1.00000

Horizonte 12 días:
  R2    -> estadístico = 3.000, p-valor = 0.99707
  RMSE  -> estadístico = 52.000, p-valor = 0.99707

Horizonte 15 días:
  R2    -> estadístico = 1.000, p-valor = 0.99902
  RMSE  -> estadístico = 54.000, p-valor = 0.99902

